In [ ]:
!pip install transformers
!pip install datasets

In [ ]:

# 데이터 로드를 위함
from datasets import load_dataset

# 기본 파이썬 패키지
import pandas as pd
import numpy as np
import datetime
from torch.optim import AdamW
from tqdm import tqdm

# GPT 사용을 위함
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# 전처리 및 평가 지표
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score


In [ ]:
!wget https://raw.githubusercontent.com/ukairia777/finance_sentiment_corpus/main/finance_data.csv


In [ ]:
df = pd.read_csv('finance_data.csv')
print('샘플의 개수 :', len(df))


In [ ]:
df.head()


In [ ]:
df['labels'].value_counts()


In [ ]:
df['labels'] = df['labels'].replace(['neutral', 'positive', 'negative'],[0, 1, 2])
df.head()


In [ ]:
df.to_csv('finance_data.csv', index=False, encoding='utf-8-sig')


In [ ]:
all_data = load_dataset(
        "csv",
        data_files={
            "train": "finance_data.csv",
        },
    )


In [ ]:
cs = all_data['train'].train_test_split(0.2, seed=777)
train_cs = cs["train"]
test_cs = cs["test"]


In [ ]:
train_cs


In [ ]:
# 훈련 데이터를 다시 8:2로 분리 후 훈련 데이터와 검증 데이터로 저장
cs = train_cs.train_test_split(0.2, seed=777)
train_cs = cs["train"]
valid_cs = cs["test"]


### Training DatasSet 추가

In [ ]:
from datasets import Dataset, concatenate_datasets

# 새 데이터
new_data = {"kor_sentence": ["경북 청도 열차사고로 사망 2명중상 4명경상 1명",
                            "속보경북 청도서 무궁화호 열차가 선로 노동자 덮쳐2명 사망5명 부상"],
            "sentence": ["", ""],
            "labels": [2, 2]}

new_dataset = Dataset.from_dict(new_data)

# 합치기
train_cs = concatenate_datasets([train_cs, new_dataset])


added_sentence = [
    "경북 청도 열차사고로 사망 2명중상 4명경상 1명",
    "속보경북 청도서 무궁화호 열차가 선로 노동자 덮쳐2명 사망5명 부상"
]
train_cs

print('두번째 샘플 출력 :', train_cs['kor_sentence'][3101])
print('두번째 샘플의 레이블 출력 :', train_cs['labels'][3101])
train_cs

In [ ]:
# 훈련 데이터
train_cs


In [ ]:
# 검증 데이터
valid_cs


In [ ]:
# 테스트 데이터
test_cs


In [ ]:
print('두번째 샘플 출력 :', train_cs['kor_sentence'][1])
print('두번째 샘플의 레이블 출력 :', train_cs['labels'][1])


# 데이터 전처리

In [ ]:
# 훈련 데이터, 검증 데이터, 테스트 데이터
train_sentences = list(train_cs['kor_sentence'])
validation_sentences = list(valid_cs['kor_sentence'])
test_sentences = list(test_cs['kor_sentence'])


In [ ]:
train_labels = train_cs['labels']
validation_labels = valid_cs['labels']
test_labels = test_cs['labels']


In [ ]:
test_sentences[:5]


In [ ]:
test_labels[:5]


# GPT 토크나이저를 이용한 전처리

In [ ]:
# 한국어 GPT 중 하나인 'skt/kogpt2-base-v2'를 사용.
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')


In [ ]:
def pad_sequences(sequences, maxlen=None, dtype='int32', padding='pre',
                  truncating='pre', value=0.0):
    """
    시퀀스들을 동일한 길이로 패딩하는 함수

    Args:
        sequences: 패딩할 시퀀스들의 리스트
        maxlen: 최대 시퀀스 길이. None이면 가장 긴 시퀀스 길이 사용
        dtype: 출력 배열의 데이터 타입
        padding: 'pre' (앞쪽 패딩) 또는 'post' (뒤쪽 패딩)
        truncating: 'pre' (앞쪽 자르기) 또는 'post' (뒤쪽 자르기)
        value: 패딩에 사용할 값

    Returns:
        패딩된 numpy 배열
    """
    if not sequences:
        return np.array([])

    # 최대 길이 결정
    if maxlen is None:
        maxlen = max(len(seq) for seq in sequences)

    # 결과 배열 초기화
    if dtype == 'long':
        result = np.full((len(sequences), maxlen), value, dtype=np.int64)
    else:
        result = np.full((len(sequences), maxlen), value, dtype=dtype)

    for i, seq in enumerate(sequences):
        if len(seq) == 0:
            continue

        # 시퀀스가 maxlen보다 긴 경우 자르기
        if len(seq) > maxlen:
            if truncating == 'pre':
                seq = seq[-maxlen:]  # 앞쪽 자르기
            else:  # truncating == 'post'
                seq = seq[:maxlen]   # 뒤쪽 자르기

        # 패딩 적용
        if padding == 'pre':
            # 앞쪽 패딩: 뒤쪽부터 채우기
            result[i, -len(seq):] = seq
        else:  # padding == 'post'
            # 뒤쪽 패딩: 앞쪽부터 채우기
            result[i, :len(seq)] = seq

    return result


In [ ]:
# 최대 길이는 128
max_len = 128

def data_to_tensor (sentences, labels, MAX_LEN):
  # 정수 인코딩 과정. 각 텍스트를 토큰화한 후에 Vocabulary에 맵핑되는 정수 시퀀스로 변환한다.
  # ex) ['안녕하세요'] ==> ['안', '녕', '하세요'] ==> [231, 52, 45]
  tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

  # pad_sequences는 패딩을 위한 모듈. 주어진 최대 길이를 위해서 뒤에서 패딩 토큰의 번호로 채워준다.
  # ex) [231, 52, 45] ==> [231, 52, 45, 패딩 토큰, 패딩 토큰, 패딩 토큰]
  pad_token = tokenizer.encode('<pad>')[0]
  input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, value=pad_token, dtype="long", truncating="post", padding="post")

  attention_masks = []

  for seq in input_ids:
      seq_mask = [float(i != pad_token) for i in seq]
      attention_masks.append(seq_mask)

  tensor_inputs = torch.tensor(input_ids)
  tensor_labels = torch.tensor(labels)
  tensor_masks = torch.tensor(attention_masks)

  return tensor_inputs, tensor_labels, tensor_masks


In [ ]:
# 학습 데이터, 검증 데이터, 테스트 데이터에 대해서
# 정수 인코딩 결과, 레이블, 어텐션 마스크를 각각 inputs, labels, masks에 저장.
train_inputs, train_labels, train_masks = data_to_tensor(train_sentences, train_labels, max_len)
validation_inputs, validation_labels, validation_masks = data_to_tensor(validation_sentences, validation_labels, max_len)
test_inputs, test_labels, test_masks = data_to_tensor(test_sentences, test_labels, max_len)


In [ ]:
# 첫 번째 샘플의 정수 인코딩 결과
sample_input = test_inputs[0]
sample_label = test_labels[0]
sample_mask = test_masks[0]

print("정수 인코딩 결과:", sample_input)
print("-" * 20)

# 원본 문장 복원
decoded_text = tokenizer.decode(sample_input)
print("원본 문장 복원 결과:", decoded_text)
print("-" * 20)

# 어텐션 마스크
print("어텐션 마스크:", sample_mask)
print("-" * 20)

# 샘플의 길이
print("샘플의 길이:", len(sample_input))
print("-" * 20)

# 레이블
print("레이블:", sample_label)


# 데이터의 배치화

In [ ]:
batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)


In [ ]:
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


In [ ]:
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)


In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')


# 모델 로드

In [ ]:
num_labels = 3

model = AutoModelForSequenceClassification.from_pretrained("skt/kogpt2-base-v2", num_labels=num_labels)
model.cuda()


# 모델 학습

In [ ]:
# 몇 번의 에포크(전체 데이터에 대한 학습 횟수)를 할 것인지 선택
epochs = 3

# 옵티마이저 선택
optimizer = AdamW(model.parameters(), lr = 2e-5)


In [ ]:
def metrics(predictions, labels):
    # predictions: 모델이 예측한 결과값들의 리스트 또는 배열
    # labels: 실제 정답 레이블들의 리스트 또는 배열

    # 예측값과 실제 레이블을 별도의 변수에 할당
    y_pred = predictions
    y_true = labels

    # 사용 가능한 메트릭들을 계산

    # 정확도 (Accuracy)
    # 전체 예측 중에서 올바르게 예측한 비율
    accuracy = accuracy_score(y_true, y_pred)

    # 매크로 평균 F1 점수 (Macro-averaged F1 Score)
    # 클래스별로 F1 점수를 계산한 후, 그 평균을 구함
    # zero_division=0 옵션은 분모가 0일 경우 0을 반환하도록 설정
    f1_macro_average = f1_score(y_true=y_true, y_pred=y_pred, average='macro', zero_division=0)

    # 마이크로 평균 F1 점수 (Micro-averaged F1 Score)
    # 전체 데이터에 대해 단일 F1 점수를 계산
    # 클래스 불균형이 심한 경우에 적합
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro', zero_division=0)

    # 가중 평균 F1 점수 (Weighted-averaged F1 Score)
    # 각 클래스의 F1 점수에 해당 클래스의 샘플 수를 가중치로 곱한 후 평균을 구함
    f1_weighted_average = f1_score(y_true=y_true, y_pred=y_pred, average='weighted', zero_division=0)

    # 계산된 메트릭 결과를 딕셔너리 형태로 리턴
    metrics = {'accuracy': accuracy,
               'f1_macro': f1_macro_average,
               'f1_micro': f1_micro_average,
               'f1_weighted': f1_weighted_average}
    return metrics


In [ ]:
def train_epoch(model, train_dataloader, optimizer, device):
    """
    하나의 에포크 동안 모델을 학습시키는 함수입니다.

    Parameters:
    model (torch.nn.Module): 학습시킬 모델 객체.
    train_dataloader (torch.utils.data.DataLoader): 학습 데이터셋의 DataLoader.
    optimizer (torch.optim.Optimizer): 최적화 알고리즘을 구현하는 객체.
    device (torch.device): 학습에 사용할 장치(CPU 또는 CUDA).

    Returns:
    float: 평균 학습 손실값.
    """

    total_train_loss = 0  # 학습 손실을 누적할 변수 초기화
    model.train()  # 모델을 학습 모드로 설정

    # 학습 데이터로더를 순회하며 배치 단위로 학습
    for step, batch in tqdm(enumerate(train_dataloader), desc="Training Batch"):
        batch = tuple(t.to(device) for t in batch)  # DataLoader에서 배치를 받아 각 텐서를 지정된 장치로 이동
        b_input_ids, b_input_mask, b_labels = batch  # 배치에서 입력 ID, 마스크, 라벨 추출

        # 모델에 배치를 전달하여 손실값 계산
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)

        # 손실값 추출
        loss = outputs.loss

        optimizer.zero_grad()  # 기울기(gradient) 초기화
        loss.backward()  # 역전파를 통해 기울기(gradient) 계산
        optimizer.step()  # 매개변수 업데이트

        total_train_loss += loss.item()  # 총 손실에 더함

    avg_train_loss = total_train_loss / len(train_dataloader)  # 평균 학습 손실 계산

    return avg_train_loss  # 평균 학습 손실 반환


In [ ]:
def evaluate(model, validation_dataloader, device):
    """
    모델을 사용하여 검증 데이터셋에 대한 평가를 수행하는 함수입니다.

    Parameters:
    model (torch.nn.Module): 평가할 모델 객체.
    validation_dataloader (torch.utils.data.DataLoader): 검증 데이터셋의 DataLoader.
    device (torch.device): 평가에 사용할 장치(CPU 또는 CUDA).

    Returns:
    float: 평균 검증 손실값.
    dict: 다양한 평가 지표(metrics)에 대한 값들을 담은 사전.
    """

    model.eval()  # 모델을 평가 모드로 설정

    total_eval_loss = 0  # 검증 손실을 누적할 변수 초기화
    predictions, true_labels = [], []  # 예측값과 실제 라벨값을 저장할 리스트 초기화

    # 검증 데이터로더를 순회하며 배치 단위로 평가
    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)  # 배치 데이터를 디바이스로 이동
        b_input_ids, b_input_mask, b_labels = batch  # 배치에서 입력 ID, 마스크, 라벨 추출

        with torch.no_grad():  # 기울기(gradient) 계산을 수행하지 않음
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)

        # 모델 출력에서 손실값 추출
        if outputs.loss is not None:
            loss = outputs.loss
            total_eval_loss += loss.item()  # 총 손실에 더함

        logits = outputs.logits.detach().cpu().numpy()  # 모델 예측값(로짓)을 numpy 배열로 변환
        label_ids = b_labels.to('cpu').numpy()  # 실제 라벨값을 numpy 배열로 변환

        # 3개의 값 중 가장 큰 값을 예측한 인덱스로 결정 (예시: logits = [3.513, -0.309, -2.111] ==> 예측: 0)
        predictions.extend(np.argmax(logits, axis=1).flatten()) # 예측된 클래스를 리스트에 추가
        true_labels.extend(label_ids.flatten()) # 실제 레이블 값을 리스트에 추가

    eval_metrics = metrics(predictions, true_labels)

    return total_eval_loss / len(validation_dataloader), eval_metrics


In [ ]:
# 최소 검증 손실 초기화
min_val_loss = float('inf')

# 메인 학습 & 평가 루프
for epoch_i in range(0, epochs):
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))

    # 학습 단계
    train_epoch(model, train_dataloader, optimizer, device)

    print("\nRunning Validation...")
    # 검증 단계
    avg_val_loss, eval_metrics = evaluate(model, validation_dataloader, device)
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Accuracy: {0:.2f}".format(eval_metrics['accuracy']))
    print("  F1 Macro: {0:.2f}".format(eval_metrics['f1_macro']))
    print("  F1 Micro: {0:.2f}".format(eval_metrics['f1_micro']))
    print("  F1 Weighted: {0:.2f}".format(eval_metrics['f1_weighted']))

    # 검증 손실이 현재까지의 최소값보다 작은 경우 체크포인트 저장
    if avg_val_loss < min_val_loss:
        print(f"Validation loss decreased ({min_val_loss:.2f} --> {avg_val_loss:.2f}).  Saving model ...")
        # 베스트 모델 저장
        torch.save(model.state_dict(), 'model_checkpoint.pt')
        # 최소 검증 손실 업데이트
        min_val_loss = avg_val_loss


# 모델 로드 및 평가

In [ ]:
# 베스트 모델 로드
model.load_state_dict(torch.load("model_checkpoint.pt"))

avg_val_loss, eval_metrics = evaluate(model, test_dataloader, device)
print("  Test Loss: {0:.2f}".format(avg_val_loss))
print("  Accuracy: {0:.2f}".format(eval_metrics['accuracy']))
print("  F1 Macro: {0:.2f}".format(eval_metrics['f1_macro']))
print("  F1 Micro: {0:.2f}".format(eval_metrics['f1_micro']))
print("  F1 Weighted: {0:.2f}".format(eval_metrics['f1_weighted']))


# 추론

In [ ]:
from transformers import pipeline

pipe = pipeline("text-classification", model=model.cuda(), tokenizer=tokenizer, device=0, max_length=512, return_all_scores=True, function_to_apply='softmax')


In [ ]:
result = pipe('SK하이닉스가 매출이 급성장하였다')
print(result)


In [ ]:
# return_all_scores 제거
pipe = pipeline("text-classification", model=model.cuda(), tokenizer=tokenizer, device=0, max_length=512, function_to_apply='softmax')

result = pipe('SK하이닉스가 매출이 급성장하였다')
print(result)


In [ ]:
label_dict = {'LABEL_0' : '중립', 'LABEL_1' : '긍정', 'LABEL_2' : '부정'}

def prediction(text):
  result = pipe(text)

  return [label_dict[result[0]['label']]]


In [ ]:
prediction('SK하이닉스가 매출이 급성장하였다 ')


In [ ]:
import pandas as pd


df = pd.read_csv('/content/news_headlines.csv', encoding='utf-8')
df = df.drop(['글자수', '크롤링시간', '순번'], axis=1)
df_head = df['헤드라인']

df['감성'] = [prediction(headline)[0] for headline in df_head]

df.head(30)